In [1]:
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import sklearn

In [2]:
df = pd.read_csv("./train.csv")

Small dataset. Will definitely be doing CV.  

  Some prelim observations:  
  GarageYrBlt: why float? only year. Check that no halves, then perhaps convert to int.  
  Lots of categorical encoding to do. For the features:  
  * if no ordinality, choose one-hot (such a small dataset, so do we need to be concerned about the extra dimensionality in this case?); not target encoding, only so I don't have to think about data leakage)
  * if ordinality, choose label

  All NA have become NaN. Don't know whether that will mean different behaviour elsewhere at all  
    
Compute correlations (R-sq) *before* training to inform feature engineering. This also possible to do in CV framework?

In [3]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [4]:
df.loc[df["GarageYrBlt"] % 1 != 0, "GarageYrBlt"].notna()

39      False
48      False
78      False
88      False
89      False
        ...  
1349    False
1407    False
1449    False
1450    False
1453    False
Name: GarageYrBlt, Length: 81, dtype: bool

In [5]:
df.loc[df["GarageYrBlt"] % 1 != 0, "GarageYrBlt"].notna().value_counts()

GarageYrBlt
False    81
Name: count, dtype: int64

In [6]:
df.loc[df["GarageYrBlt"] % 1 != 0, ["GarageYrBlt", "GarageArea"]]

,GarageYrBlt,GarageArea
39,NaN,0
48,NaN,0
78,NaN,0
88,NaN,0
89,NaN,0
...,...,...
1349,NaN,0
1407,NaN,0
1449,NaN,0
1450,NaN,0


&#8595; &#8595; scenario where df[df... will not work: only df.loc[df

In [7]:
df.loc[df["PoolQC"].notna(), "PoolQC"]

197     Ex
810     Fa
1170    Gd
1182    Ex
1298    Gd
1386    Fa
1423    Gd
Name: PoolQC, dtype: object

In [8]:
# Start by one-hot encoding MSZoning, Street, Alley
# At least a few categorical columns suitable for label encoding -- ordinality
df_onehot = pd.get_dummies(df, columns=["MSZoning", "Street", "Alley"])
df_onehot

,Id,MSSubClass,LotFrontage,LotArea,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,SalePrice,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave
0,1,60,65.0,8450,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,208500,False,False,False,True,False,False,True,False,False
1,2,20,80.0,9600,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,181500,False,False,False,True,False,False,True,False,False
2,3,60,68.0,11250,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,223500,False,False,False,True,False,False,True,False,False
3,4,70,60.0,9550,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,140000,False,False,False,True,False,False,True,False,False
4,5,60,84.0,14260,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,250000,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,...,175000,False,False,False,True,False,False,True,False,False
1456,1457,20,85.0,13175,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,...,210000,False,False,False,True,False,False,True,False,False
1457,1458,70,66.0,9042,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,...,266500,False,False,False,True,False,False,True,False,False
1458,1459,20,68.0,9717,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,142125,False,False,False,True,False,False,True,False,False


In [9]:
# Why in above output does MSZoning C column have "all"?
df.loc[(df["MSZoning"]=="A") | (df["MSZoning"]=="I") | (df["MSZoning"]=="RP")]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


Note syntax above: parentheses after df.loc[  , NOT after df[  ; the pipe

In [10]:
from sklearn.feature_selection import r_regression
r_sq_score = r_regression(df_onehot[["LotArea"]], df_onehot["SalePrice"]) ** 2
r_sq_score 

array([0.06961332])

In [11]:
# Computing r-squared with r_regression on one-hot encoded feature doesn't really work, 
# because you get separate coefficient for each column
zonecolumns = df_onehot[["MSZoning_C (all)", "MSZoning_FV", "MSZoning_RH", "MSZoning_RL", "MSZoning_RM"]]
r_sq_score_zoning = r_regression(zonecolumns, df_onehot["SalePrice"]) ** 2
r_sq_score_zoning

array([0.01237803, 0.00809095, 0.00428099, 0.06005607, 0.08298151])

In [12]:
# Need to fit LinReg model instead: then can compute r-squared on all columns together
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
X = df_onehot[df_onehot.columns[df_onehot.columns.str.startswith("MSZoning")]]
y = df_onehot["SalePrice"]
model = LinearRegression()
model.fit(X, y) 
y_pred = model.predict(X)
r2_score(y, y_pred)

0.10755968344624967

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
X = df_onehot[["LotArea"]]
y = df_onehot["SalePrice"]
model = LinearRegression()
model.fit(X, y) 
y_pred = model.predict(X)
r2_score(y, y_pred)

0.0696133153821119

In [14]:
# Don't think there is a need to scale one-hot encoded features for LinReg.
df_onehot = pd.get_dummies(df_onehot, columns=["Neighborhood"])
df_onehot

,Id,MSSubClass,LotFrontage,LotArea,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,1,60,65.0,8450,Reg,Lvl,AllPub,Inside,Gtl,Norm,...,False,False,False,False,False,False,False,False,False,False
1,2,20,80.0,9600,Reg,Lvl,AllPub,FR2,Gtl,Feedr,...,False,False,False,False,False,False,False,False,False,True
2,3,60,68.0,11250,IR1,Lvl,AllPub,Inside,Gtl,Norm,...,False,False,False,False,False,False,False,False,False,False
3,4,70,60.0,9550,IR1,Lvl,AllPub,Corner,Gtl,Norm,...,False,False,False,False,False,False,False,False,False,False
4,5,60,84.0,14260,IR1,Lvl,AllPub,FR2,Gtl,Norm,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,Reg,Lvl,AllPub,Inside,Gtl,Norm,...,False,False,False,False,False,False,False,False,False,False
1456,1457,20,85.0,13175,Reg,Lvl,AllPub,Inside,Gtl,Norm,...,False,False,False,False,False,False,False,False,False,False
1457,1458,70,66.0,9042,Reg,Lvl,AllPub,Inside,Gtl,Norm,...,False,False,False,False,False,False,False,False,False,False
1458,1459,20,68.0,9717,Reg,Lvl,AllPub,Inside,Gtl,Norm,...,False,False,False,False,False,False,False,False,False,False


In [15]:
df_onehot[df_onehot.columns[df_onehot.columns.str.startswith("Neighborhood")]]

,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1456,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1457,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1458,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [16]:
df["Neighborhood"].unique()

array(['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst',
       'NWAmes', 'OldTown', 'BrkSide', 'Sawyer', 'NridgHt', 'NAmes',
       'SawyerW', 'IDOTRR', 'MeadowV', 'Edwards', 'Timber', 'Gilbert',
       'StoneBr', 'ClearCr', 'NPkVill', 'Blmngtn', 'BrDale', 'SWISU',
       'Blueste'], dtype=object)

In [17]:
df[["Neighborhood", "SalePrice"]]

,Neighborhood,SalePrice
0,CollgCr,208500
1,Veenker,181500
2,CollgCr,223500
3,Crawfor,140000
4,NoRidge,250000
...,...,...
1455,Gilbert,175000
1456,NWAmes,210000
1457,Crawfor,266500
1458,NAmes,142125


In [18]:
X = df_onehot[df_onehot.columns[df_onehot.columns.str.startswith("Neighborhood")]]
y = df_onehot["SalePrice"]
model = LinearRegression()
model.fit(X, y) 
y_pred = model.predict(X)
r2_score(y, y_pred)

0.5455749908095633

In [19]:
df_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 111 entries, Id to Neighborhood_Veenker
dtypes: bool(34), float64(3), int64(35), object(39)
memory usage: 926.9+ KB


In [20]:
query = df_onehot[df_onehot.columns[df_onehot.columns.str.startswith("Bsmt")]]

In [21]:
len(query.loc[query["BsmtQual"].isna()])

37

In [22]:
df_onehot.select_dtypes(include='object')

,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,2Story,Gable,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,Reg,Lvl,AllPub,FR2,Gtl,Feedr,Norm,1Fam,1Story,Gable,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,2Story,Gable,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,2Story,Gable,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,IR1,Lvl,AllPub,FR2,Gtl,Norm,Norm,1Fam,2Story,Gable,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,2Story,Gable,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1456,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,1Story,Gable,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1457,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,2Story,Gable,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal
1458,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,1Story,Hip,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal


Features still to one-hot encode: MSSubClass, LandContour, LotConfig, BldgType, HouseStyle, RoofStyle, RoofMatl,  
  
  LEAVE OUT Exterior1st and ...2nd for now because we need to combine them first; AND Cond 1 and 2, 

list continued: Foundation, Heating, CentralAir, Electrical, PavedDrive, SaleType, SaleCondition

    

In [27]:
df_onehot = pd.get_dummies(df_onehot, columns=["MSSubClass", "LandContour", "LotConfig", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl",
                                       "Foundation", "Heating", "CentralAir", "Electrical", "PavedDrive", "SaleType", "SaleCondition"])

In [37]:
len(df_onehot.loc[df_onehot["Condition1"] != df_onehot["Condition2"]])

195

In [39]:
len(df_onehot["Condition1"].value_counts()) == len(df_onehot["Condition2"].value_counts())

False

In [57]:
f"""Condition1 column has {len(df_onehot["Condition1"].value_counts())} distinct values. \
Condition2 column has {len(df_onehot["Condition2"].value_counts())}  distinct values."""

'Condition1 column has 9 distinct values. Condition2 column has 8  distinct values.'

In [43]:
df_onehot["Condition2"].value_counts()

Condition2
Norm      1445
Feedr        6
Artery       2
RRNn         2
PosN         2
PosA         1
RRAn         1
RRAe         1
Name: count, dtype: int64

In [23]:
# In some cases the same material is spelt differently in 1st column vs in 2nd column. Not good.
# So I will change the names of those ones so that they match
df_onehot.loc[df_onehot["Exterior1st"] != df_onehot["Exterior2nd"], ["Exterior1st", "Exterior2nd"]]


,Exterior1st,Exterior2nd
3,Wd Sdng,Wd Shng
8,BrkFace,Wd Shng
11,WdShing,Wd Shng
12,HdBoard,Plywood
19,BrkFace,Plywood
...,...,...
1416,VinylSd,AsbShng
1448,MetalSd,HdBoard
1449,CemntBd,CmentBd
1451,CemntBd,CmentBd


In [24]:
df_onehot["Exterior1st"].value_counts()

Exterior1st
VinylSd    515
HdBoard    222
MetalSd    220
Wd Sdng    206
Plywood    108
CemntBd     61
BrkFace     50
WdShing     26
Stucco      25
AsbShng     20
BrkComm      2
Stone        2
AsphShn      1
ImStucc      1
CBlock       1
Name: count, dtype: int64

In [25]:
df_onehot["Exterior2nd"].value_counts()

Exterior2nd
VinylSd    504
MetalSd    214
HdBoard    207
Wd Sdng    197
Plywood    142
CmentBd     60
Wd Shng     38
Stucco      26
BrkFace     25
AsbShng     20
ImStucc     10
Brk Cmn      7
Stone        5
AsphShn      3
Other        1
CBlock       1
Name: count, dtype: int64

In [58]:
f"""Exterior1st column has {len(df_onehot["Exterior1st"].value_counts())} distinct values. \
Exterior2nd column has {len(df_onehot["Exterior2nd"].value_counts())} distinct values."""

'Exterior1st column has 15 distinct values. Exterior2nd column has 16 distinct values.'

It's surely overcomplicating to have two columns so similar to each another. I will try to merge them 